In [ ]:
# packages
#pip install neo4j-driver
#pip install openpyxl
#pip install itertools

In [15]:
# imports
from neo4j import GraphDatabase, basic_auth
import pandas as pd
from openpyxl import Workbook, load_workbook
from itertools import islice

In [2]:
# set driver connection
driver = GraphDatabase.driver(
  "neo4j://107.20.12.116",
  auth=basic_auth("neo4j", "bond-scream-transistors"))

In [3]:
# define query functions
def name_query(name):
    cypher_query = '''
    MATCH (a:Officer {name:$name})-[r:officer_of|intermediary_of|registered_address*..10]-(b)
    RETURN b.name as name, b.node_id as node_id LIMIT 20
    '''
    with driver.session(database="neo4j") as session:
        results = session.execute_read(lambda tx: tx.run(cypher_query, name=name).data())
        print(f'{name}: {len(results)} results')
        driver.close()


def city_query(city):
    cypher_query = f'''
    MATCH (b)
    WHERE toLower(b.name) CONTAINS toLower(" {city} ")
    RETURN  *
    '''
    with driver.session(database="neo4j") as session:
        results = session.execute_read(lambda tx: tx.run(cypher_query).data())
        print(f'{city}: {len(results)} results')
        driver.close()

In [28]:
# import excel file (.xlsx)
book = load_workbook("Rechercheliste_Niedersachsen.xlsx")
sheet = book.active

data = sheet.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)

# convert worksheet to a Dataframe
df = pd.DataFrame(data, index=idx, columns=cols)
df.head()

,Geburtsdatum,Wohnort (vmtl.),Firmensitz,NaN,NaN,NaN
Martin Kind,1944-04-28 00:00:00,Burgwedel,None,None,None,None
Kind GmbH & Co. KG,None,None,"Kokenhorststraße 3-5, 30938 Großburgwedel",None,None,None
Dirk Roßmann,1976-09-07 00:00:00,"Wittenbecksweg 20, 29646 Bispingen",None,None,None,None
Raoul Roßmann,1986-08-10 00:00:00,"Blücherstr. 5, 30175 Hannover",None,None,None,None
Dirk Daniel Roßmann,1976-07-23 00:00:00,"Riethof 6 B, 30916 Isernhagen",None,None,None,None


In [5]:
# run query for names
query_list_names = ["BERLUSCONI"] # list of names
for name in query_list_names:
    name_query(name)

BERLUSCONI: 0 results


In [6]:
# run query for cities
query_list_cities = ["Bremen"] # list of cities
for city in query_list_cities:
    city_query(city)

Bremen: 31 results
